# Classifiers comparison on texts with naive Bayes assumption

In this session of laboratory we compare two models for categorical data probabilistic modeling: 
1. multivariate Bernoulli 
2. multinomial on a dataset 

We adopt a dataset on Twitter messages labelled with emotions (Joy vs Sadness).

The following program shows the loading of the data from a file.


Data are loaded into a matrix X adopting a sparse matrix representation, in order to save space and time.
Sparse matrix representation (in the csr format) represents in three "parallel" arrays the value of the matrix cells that are different from zero and the indices of those matrix cells.
The arrays are called: 
* data
* row
* col

* data[i] stores the value of the matrix cell #i whose indexes are contained in row[i] and col[i] 
* row[i] stores the index of the row in the matrix of the cell #i, 
* col[i] stores the index of the column of the cell #i.


The data file is in csv format.
Any Twitter message has been preprocessed by a Natural Language pipeline which eliminated stop words and substituted the interesting document elements with an integer identifier.  
The interesting document elements might be words, emoji or emoticons. The elements could be repeated in the same document and are uniquely identified in the documents by the same integer number (named "element_id" in the program). This "element_id" number will be used as the index of the column of the data matrix, for the purposes of storage of data.

Each row of the CSV file reports the content of a document (a Twitter message). It is formed as a list of integer number pairs, followed by a string which is the label of the document ("Joy" or "Sadness").
The first number of the pair is the identifier of a document element (the "element_id"); 
the second number of the pair is the count (frequency) of that element in that document.

The dataset has:

tot_n_docs (or rows in the file) =n_rows=11981

n_features (total number of distinct words in the corpus)=11288


The following program reads the data file and loads in a sparse way the matrix using the scipy.sparse library


In [43]:
from numpy import ndarray, zeros
import numpy as np
import scipy
from scipy.sparse import csr_matrix

class_labels = ["Joy", "Sadness"]
n_features = 11288  # number of columns in the matrix = number of features (distinct elements in the documents)
n_rows = 11981  # number rows of the matrix
n_elements = 71474  # number of the existing values in the matrix (not empty, to be loaded in the matrix in a sparse way)

# path_training="/Users/meo/Documents/Didattica/Laboratorio-15-16-Jupyter/"
path_training = "./"
file_name = "joy_sadness6000.txt"

# declare the row and col arrays with the indexes of the matrix cells (non empty) to be loaded from file
# they are needed because the matrix is sparse and we load in the matrix only the elements which are present
row = np.empty(n_elements, dtype=int)
col = np.empty(n_elements, dtype=int)
data = np.empty(n_elements, dtype=int)

row_n = 0
cur_el = 0
twitter_labels = []
twitter_target = []  # list of 0/1 for class labels
with open(path_training + file_name, "r") as fi:
    for line in fi:
        el_list = line.split(",")
        l = len(el_list)
        last_el = el_list[l - 1]
        class_name = last_el.strip()
        twitter_labels.append(class_name)
        if class_name == class_labels[0]:
            twitter_target.append(0)
        else:
            twitter_target.append(1)
        i = 0
        while i < (l - 1):
            element_id = int(el_list[i])
            element_id = element_id - 1
            i = i + 1
            value_cell = int(el_list[i])
            i = i + 1
            row[cur_el] = row_n
            col[cur_el] = element_id
            data[cur_el] = value_cell
            cur_el = cur_el + 1
        row_n = row_n + 1
fi.close
print("final n_row=" + str(row))
# loads the matrix by means of the indexes and the values in the three arrays just filled
twitter_data = csr_matrix((data, (row, col)), shape=(n_rows, n_features)).toarray()
print("resulting matrix:")
print(twitter_data)
print(twitter_labels)
print(twitter_target)

final n_row=[0 0 0 ... 0 0 0]
resulting matrix:
[[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]
['Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy'

Write a program in the following cell that splits the data matrix in training and test set (by random selection) and predicts the class (Joy/Sadness) of the messages on the basis of the words. 
Consider the two possible models:
multivariate Bernoulli and multinomial Bernoulli.
Find the accuracy of the models and test is the observed differences are significant.


# Fitting

I fit the models using cross validation


In [44]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.model_selection import cross_validate, train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    twitter_data, twitter_target, test_size=0.25, random_state=42, shuffle=True
)

multivariate_bernoulli_model = BernoulliNB(force_alpha=True)
multinomial_bernoulli_model = MultinomialNB(force_alpha=True)
multivariate_bernoulli_model.fit(X_train, y_train)
multinomial_bernoulli_model.fit(X_train, y_train)

multivariate_scores = cross_validate(
    multivariate_bernoulli_model, twitter_data, twitter_target, scoring="f1", cv=20
)
multinomial_scores = cross_validate(
    multinomial_bernoulli_model, twitter_data, twitter_target, scoring="f1", cv=20
)
print(multivariate_scores)
print(multinomial_scores)

{'fit_time': array([3.73218822, 3.68243194, 3.65244699, 4.07200003, 3.62804604,
       3.61790419, 3.6149559 , 3.62903285, 3.62380695, 3.62096715,
       3.88631511, 3.78995299, 3.62710214, 3.635221  , 3.85820103,
       3.63153982, 3.62088871, 3.61328602, 3.63201094, 3.63874793]), 'score_time': array([0.05346489, 0.05171299, 0.04947472, 0.05900288, 0.05264401,
       0.04787374, 0.04777408, 0.04966998, 0.05007005, 0.05017877,
       0.04836988, 0.05467105, 0.05386686, 0.06556106, 0.04765797,
       0.04766107, 0.04999328, 0.04921389, 0.0526669 , 0.04713821]), 'test_score': array([0.95723684, 0.96078431, 0.96880131, 0.96039604, 0.93548387,
       0.96369637, 0.9472    , 0.96039604, 0.95961228, 0.94041868,
       0.95315024, 0.95765472, 0.95284553, 0.95253682, 0.95253682,
       0.95666132, 0.96880131, 0.96103896, 0.93851133, 0.9398374 ])}
{'fit_time': array([2.82667804, 2.8209312 , 2.81309581, 2.8324523 , 2.80303407,
       2.85873008, 2.82027864, 2.826998  , 2.80915189, 2.80679584,
  

# Predictions

I want a confusion matrix of the 2 models
I also show the F1_scores


In [45]:
from sklearn.metrics import confusion_matrix, f1_score

mv_y_pred = multivariate_bernoulli_model.predict(X_test)
mv_confusion_matrix = confusion_matrix(y_test, mv_y_pred)
print(f"Multivariate Confusion Matrix:\n{mv_confusion_matrix}")
print(f"The Multivariate F_1 Score:\t{f1_score(y_test, mv_y_pred)}")

mn_y_pred = multinomial_bernoulli_model.predict(X_test)
mn_confusion_matrix = confusion_matrix(y_test, mn_y_pred)
print(f"Multinomial Confusion Matrix:\n{mn_confusion_matrix}")
print(f"The Multinomial F_1 Score:\t{f1_score(y_test, mn_y_pred)}")

Multivariate Confusion Matrix:
[[1405  120]
 [  28 1443]]
The Multivariate F_1 Score:	0.9512195121951219
Multinomial Confusion Matrix:
[[1396  129]
 [  28 1443]]
The Multinomial F_1 Score:	0.9484061781137035


# Hypothesis Test

I will now check if the two models are statistically the same:
Using cross-validation I extract a metric
Using a Student's T test I test if the difference of the metric is 0 (the models perform identically)


In [46]:
from scipy.stats import ttest_1samp

score_differences = [
    mn_s - mv_s
    for mn_s, mv_s in zip(
        multinomial_scores["test_score"], multivariate_scores["test_score"]
    )
]
print(score_differences)

mu = np.average(score_differences)
print(f"mean of observations: {mu}")
# popmean is the mean of the student's t (we put it to 0 because we have H_0: The models are the same model)
# alternative is saying that in the alternative hypothesis H_1 we do not care if the mean is greater or less we just consider it different
g = ttest_1samp(score_differences, popmean=0, alternative="greater")
l = ttest_1samp(score_differences, popmean=0, alternative="less")
print(g)
print(l)

[-0.0015718174747213487, -0.004686752749880396, -0.0013746035311799076, -0.00013113894170890816, -0.004727027167419795, 0.005002147661224199, -0.003021371610845369, -0.0014469427238291699, -0.00168347527853141, -0.004934808581372518, -0.006462035541195399, -0.0015571621514260947, 0.00015285079917248812, -0.0029433289422909903, -0.0013967597306705493, -0.004738239288801127, -0.001481052190991261, -0.0031101584499642065, 0.0001983505585134715, 0.0005812811751305658]
mean of observations: -0.0019666022080393865
TtestResult(statistic=-3.4387727332702913, pvalue=0.9986239815760719, df=19)
TtestResult(statistic=-3.4387727332702913, pvalue=0.0013760184239280779, df=19)


# Results

## Using f1_score and 20 fold cross validation

The mean of the dataset is $\mu = -0.002$ (slightly favours the multivariate model as it is negative)

Be reminded that t-test works with means (averages of data) so we assume the mean of the dataset should be extracted from a random variable $\Mu$ with mean $\mu_0 = 0$ (as set in popmean).

We got:

* $P(\Mu \ge \mu | H_0) = 99.86 \%$
* $P(\Mu \le \mu | H_0) = 0.14 \%$

Therefore we are in the *LEFT* side of the distribution and the result is too extreme on the left for the hypothesis to be correct. We then have to refuse the $H_0$ hypothesis in favour of one with a lower mean ( $\mu_0$ ). 
We can say with $99.86\%$ certainty (confidence level) that the multivariate model performs better.
